In [ ]:
%load_ext autoreload
%autoreload 2

# Finishing the Handshake

Take a peek at [where we left off last time](../2.%20Reading%20Version%20Messages/Lesson.ipynb#Putting-it-all-together).

We were able to entice a response from our peer and then interpret it. However, three problems remain:
1. Our initial `version` message payload is hardcoded. We should be able to construct it using any parameters we like.
2. After receiving our peer's `version` response, we don't listen for their `verack` response as the [version handshake](https://en.bitcoin.it/wiki/Version_Handshake) says we should.
3. We don't send our `verack` upon receipt of our peer's `verack`, the final step in the handshake.

Once we fix all these problems our program will be able to join the Bitcoin peer-to-peer network just like a [Bitcoin Core](https://github.com/bitcoin/bitcoin) full node does.  We won't be able to participate nearly as fully or effectively as Bitcoin Core node, but it's a start!

The last 2 problems are easy to fix. `verack` messages are easy to serialize and deserialize because they [have no payload](https://en.bitcoin.it/wiki/Protocol_documentation#verack). 

Problem #1 will be more involved. We've spent a lot of time learning to _deserialize_ Bitcoin network messages: to turn raw bytes into Python objects. Problem #1 demands we do the opposite: _serialize_ Bitcoin messages, turn Python objects into raw bytes that can be sent over the network to our peers, who may not even have Python installed!

If you've ever done any web development I'm sure you've learned to serialize and deserialize JSON, which is the de facto data representation on the web. Same thing here, but we're usng bytes instead of JSON.

Let's tackle these problems one-by-one.

# Problem #1: Constructing Version Messages

We desire `serialize_version_payload` and `serialize_message` functions which could be used like this:

```python
version_payoad = serialize_version_payload(
    version=7011,
    user_agent=b'/buidl-army/',
)
msg = serialize_msg(command=b'verack', payload=version_payload)
sock.send(msg)
```

These two functions would allow us to compose version messages to our exact desires. And in the future we will re-use the skills we develop writing these two functions to compose any other Bitcoin protocol messages.

`serialize_version_payload` would do the exact opposite from the `read_version_payload` we wrote earlier. The `serialize_message` function would do the opposite of [`read_message`](http://localhost:8889/notebooks/2.%20Reading%20Version%20Messages/Lesson.ipynb#Housekeeping) function we wrote earlier. Now that we learned to do all these `bytes` -> Python data conversions, hopefully the inverse Python -> `bytes` data conversions will be a little easier!

So which should we implement first? Let's do `serialize_version_payload` because these ideas are fresher in our minds.

# `serialize_version_payload`

[This file](/edit/3.%20Composing%20Version%20Messages/exercises.py)  contains stubs of all the code we will write in this lesson. For every exercise you will modify this file and the unittests will import and test these functions.

Contained in that file is a stub for `serialize_version_payload`. 

* It has default parameters for every field in a a version message. 
* We need the `if` statements for `timestamp` and `nonce` because the defaults for these values need to be computed when the function is actually run and not when the code is first executed -- `timestamp` should be current unix time when the function is called, `nonce` should be a different random 8-byte number every time the function is called.
* Finally, we stick the correct number of zero bytes (`b'\x00'`) for each field as a placeholder.

As you can see this code runs but doesn't do anything useful yet:

In [ ]:
from io import BytesIO
from exercises import serialize_version_payload
from lib import read_version_payload

version_payload_bytes = serialize_version_payload()
print('Serialized payload:')
print(version_payload_bytes)

version_payload_dict = read_version_payload(BytesIO(version_payload_bytes))
print()
print('Deserialized payload:')
print(version_payload_dict)

In this lesson we will go field-by-field through this function and replace the zero bytes with the `bytes` serialization of that field.

Let's start with the integer fields because there so many of them and because they're simple.

# Integer Fields

`b\x01` is one way to encode the integer `1` as `bytes` -- in a single byte.

But that's not the only way to encode the integer `1` as `bytes`.

`b\x01\x00` is another way -- in 2 bytes with little-endian order. 

`b\x00\x00\x01` is yet another way -- in 3 bytes with big-endian order.

We can always pad the encoding with zero-bytes at higher orders-of-magnitude.

This ability is essential because it allows for fields with fixed size, and fixed field sizes are one of the ways we make sense of the endless string of bytes we receive from our peers. 

While there are benefits to fixed field sizes, there are also dangers. We must allow for room for the fields to grow into the future. For example, timestamps increase over time. We need to make room for them to grow for the many centuries we hope Bitcoin is active.

This is how you'd calculate the current unix time:

In [ ]:
import time

now = int(time.time())
now

Some `timestamp` fields in the bitcoin protocol are only 4 bytes. These require no padding:

In [ ]:
now.to_bytes(4, 'little')

But eventually these fields will start overflowing. Here I calculate the highest Unix timestamp which can be serialized in only 4 bytes -- `FF FF FF FF`. Then I show the date it corresponds to, which is the date these 4 byte Unix timestamp fields will overflow.

In [ ]:
from datetime import datetime

# The biggest 4-byte little endian number: 256**4
n_bytes = b'\xff' * 4
n = int.from_bytes(n_bytes, 'little')
print('The biggest 4 byte Unix timestamp is', n)

overflow_date = datetime.fromtimestamp(n+1)
days_from_now = (overflow_date - datetime.now()).days
print(f'4 byte timestamp fields in the Bitcoin protocol will overflow '
      f'at {overflow_date}, {days_from_now} days from now')


This is a big problem and will most likely require a hard fork in the distant future. One of Satoshi's bigger screwups for sure!

Other `timestamp` fields, however, are 8 bytes. This is much better. They currently have four bytes of padding and have lots of space to grow in the future:

In [ ]:
print('Much zero-byte padding, so growing room: ')
print(now.to_bytes(8, 'little'))

In fact, this range goes so high that your computer probably can't even interpret the high end as a timestamp:

In [ ]:
# The biggest 8-byte little endian number: 256**8
n_bytes = b'\xff' * 8
n = int.from_bytes(n_bytes, 'little')
print('The biggest 8 byte Unix timestamp is', n)

In [ ]:
# On my machine this date is so far into the future that it can't 
overflow_date = datetime.fromtimestamp(n+1)
print('8-byte timestamp fields in Bitcoin will overflow on', overflow_date)

8 byte Unix timetamp fields won't overflow for *billions* of years!

In [ ]:
secs_in_year = 365 * 24 * 60 * 60

years = n // secs_in_year

print(f'8-byte timestamp fields in Bitcoin will overflow in the year {1970 + years:,d}')

Now that's what I call breathing room!

Anyway, hopefully now you see why it's nice to have fixed-length fields with some room to grow into the future. 

### Exercise: `int_to_little_endian(int, size)` and `int_to_big_endian(int, size)` 

What does all this mean for our code?

In order to deal with such fields, our `int_to_little_endian` function needs to take 2 arguments -- the integer we will encode and the number of bytes our serialization should occupy.

Note: There is a [notebook containing the answers](Answers.ipynb) in case you get stuck

Implement `int_to_little_endian` in the [`exercises.py`](/edit/3.%20Composing%20Version%20Messages/exercises.py). When you think you have it, save the file ("control-s" or "File > save") and hope back over here to run the test in the next cell.

In [ ]:
from exercises import int_to_little_endian

def test_int_to_little_endian():
    integer = 22
    bytes = b'\x16\x00\x00\x00\x00\x00\x00\x00\x00\x00'
    result = int_to_little_endian(integer, 10)
    assert bytes == result, f'Correct answer: {bytes}. Your answer: {result}'
    print("Test passed!")

test_int_to_little_endian()

Now give big-endian serialization a try:

In [ ]:
from exercises import int_to_big_endian

def test_int_to_big_endian():
    integer = 22
    bytes = b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x16'
    result = int_to_big_endian(integer, 10)
    assert bytes == result, f'Correct answer: {bytes}. Your answer: {result}'
    print("Test passed!")

test_int_to_big_endian()

### Exercise: Update `serialize_version_payload` to serialize all integer fields integers using `int_to_little_endian` and `int_to_big_endian`

`version`, `services`, `timestamp`, `nonce`, and `start_height` are the fields we're interested in here.

Open the [`exercises.py`](/edit/3.%20Composing%20Version%20Messages/exercises.py), edit and save the code in there to get this test passing.

In [ ]:
from io import BytesIO
from lib import read_version_payload

def test_serialize_version_payload_integers():
    now = int(time.time()) - 10
    version_payload_dict = serialize_version_payload(
        services=3, nonce=4, timestamp=now, start_height=50)
    version_payload = read_version_payload(BytesIO(version_payload_dict))    
    assert version_payload['version'] == 70015
    assert version_payload['services'] == 3
    assert version_payload['timestamp'] == now
    assert version_payload['nonce'] == 4
    assert version_payload['start_height'] == 50
    print('Test passed!')
    
test_serialize_version_payload_integers()

## Services

It's all well and good that we can serialize integers. But we still don't really know how to produce the integers themselves in one case: `services`

Let's say I want to set my services bitfield to only offer support for `NODE_NETWORK` and `NODE_BLOOM`. It would currently require some tricky calculations by hand. Let's write a function that will take care of this for us:

### Exercise: `services_dict_to_int`

Here's how I would do this. I'd make a dictionary mapping `services_dict` keys to the the value of the corresponding bits:

```python
    key_to_multiplier = {
        'NODE_NETWORK': 2**0,
        'NODE_GETUTXO': 2**1,
        'NODE_BLOOM': 2**2,
        'NODE_WITNESS': 2**3,
        'NODE_NETWORK_LIMITED': 2**10,
    }
```

Then I'd loop over the keys and values in `services_dict` (`for key, value in services_dict.items()`) and sum up the `services` integers with the assistance of `key_to_multiplier`.

In [ ]:
from exercises import services_dict_to_int

def test_services_dict_to_int():
    services_dict = {
        'NODE_NETWORK': True,
        'NODE_GETUTXO': False,
        'NODE_BLOOM': True,
        'NODE_WITNESS': False,
        'NODE_CASH': True,
        'NODE_NETWORK_LIMITED': True,
    }
    answer = 1 + 4 + 1024
    result = services_dict_to_int(services_dict)
    assert answer == result,\
        f'services_dict_to_int({repr(services_dict)}) should equal {answer}, was {result}'
    print('Test passed!')
    
test_services_dict_to_int()

### Exercise: Update `serialize_version_payload` to take `services_dict` argument

Replace the `services` argument with a `services_dict` argument.

Then use `services_dict_to_int` within the body of the function to convert `services_dict` into an integer. BTW give `services_dict` a default value of the empty dictionary `{}`

In [ ]:
from exercises import serialize_version_payload

def test_serialize_version_payload_services_dict():
    now = int(time.time()) - 10
    version_payload = serialize_version_payload(
        services_dict={'NODE_NETWORK': True, 'NODE_BLOOM': True}, 
        nonce=4, timestamp=now, start_height=50)
    version_payload = read_version_payload(BytesIO(version_payload))
    assert version_payload['version'] == 70015
    assert version_payload['services'] == 5
    assert version_payload['timestamp'] == now
    assert version_payload['nonce'] == 4
    assert version_payload['start_height'] == 50
    print('Test passed!')
    
test_serialize_version_payload_services_dict()

# Boolen Fields

### Exercise: `bool_to_bytes(bool)`

Interpret a boolean as bytes

In [ ]:
from exercises import bool_to_bytes

def test_bytes_to_bool():
    assert bool_to_bytes(True) == b'\x01', \
        f'bool_to_bytes(False) should equal b"\\x01", was {bool_to_bytes(True)}'
    assert bool_to_bytes(False) == b'\x00',\
        f'bool_to_bytes(False) should equal b"\\x00", was {bool_to_bytes(False)}'
    print('Test passed!')
    
test_bytes_to_bool()

### Exercise: Update `serialize_version_payload` to call `bool_to_bytes` where appropriate

In [ ]:
from exercises import serialize_version_payload

def test_serialize_version_payload_services_dict():
    now = int(time.time()) - 10
    version_payload = serialize_version_payload(
        services_dict={'NODE_NETWORK': True, 'NODE_BLOOM': True}, 
        nonce=4, timestamp=now, start_height=50, user_agent=b'/buidl-army/')
    version_payload = read_version_payload(BytesIO(version_payload))
    assert version_payload['version'] == 70015
    assert version_payload['services'] == 5
    assert version_payload['timestamp'] == now
    assert version_payload['nonce'] == 4
    assert version_payload['start_height'] == 50
    assert version_payload['user_agent'] == b'/buidl-army/'
    assert version_payload['relay'] is True
    print('Test passed!')
    
test_serialize_version_payload_services_dict()

# Variable Length Fields

Next comes the `user_agent` field of type `varstr`.

Remember how `varstr` is itself composed of 2 fields, a `varint` followed n raw bytes when n is the value of the `varint` field?

![image](../images/varstr.png)

Therefore `serialize_varstr` will need to call a `serialize_varint` function. So let's implement `serizlize_varint` first.

### Exercise: `serialize_varint`

Recall the algorithm for varint serialization of some number `n`:

* If `n` is less than 253, return `n` as a single byte
* Else if `n` can fit in 2 bytes (less than 256**2) return `b'fd'`, the marker for a 2 byte varint, followed by `n` serialized in 2 little-endian bytes. 
* Else if `n` can fit in 4 bytes (less than 256**4) return `b'fe'`, the marker for a 4 byte varint, followed by `n` serialized in 4 little-endian bytes. 
* Else if `n` can fit in 8 bytes (less than 256**8) return `b'ff'`, the marker for an 8 byte varint, followed by `n` serialized in 8 little-endian bytes. 

Try to implement this. 

If you want to play around with a finished version of the function to get a better sense for how it works you can make a new cell and import this one: `from lib import serialize_varint`

In [ ]:
from exercises import serialize_varint

def test_serialize_varint():
    from lib import serialize_varint as _serialize_varint 
    numbers = [10+256**i for i in [0, 1, 3, 7]]
    for i in numbers:
        expected = _serialize_varint(i)
        result = serialize_varint(i)
        assert expected == result,\
            f'serialize_varint({i}) should return {expected}, returned {result}'

    print('Test passed!')

test_serialize_varint()

### Exercise: `serialize_varstr(bytes)`

There are two steps here: 
* Calculate the `varint` serialization of the length of the bytes
* Return this varint concatenated with the bytes themselves

In [ ]:
def test_serialize_varstr():
    from lib import serialize_varstr as _serialize_varstr 
    bytestrings = [
        b"hodl",
        b"A purely peer-to-peer version of electronic cash would allow online payments to be sent directly from one party to another without going through a financial institution. Digital signatures provide part of the solution, but the main benefits are lost if a trusted third party is still required to prevent double-spending. We propose a solution to the double-spending problem using a peer-to-peer network.  The network timestamps transactions by hashing them into an ongoing chain of hash-based proof-of-work, forming a record that cannot be changed without redoing the proof-of-work. The longest chain not only serves as proof of the sequence of events witnessed, but proof that it came from the largest pool of CPU power. As long as a majority of CPU power is controlled by nodes that are not cooperating to attack the network, they'll generate the longest chain and outpace attackers. The network itself requires minimal structure. Messages are broadcast on a best effort basis, and nodes can leave and rejoin the network at will, accepting the longest proof-of-work chain as proof of what happened while they were gone.",
    ]
    
    for b in bytestrings:
        answer = _serialize_varstr(b)
        result = serialize_varstr(b)
        assert result == answer,\
            f'serialize_varstr({b}) should be {answer}, was {result}'
    
    print('Test passed!')
    
test_serialize_varstr()

### Exercise: Update `serialize_version_payload` to serialize the `user_agent` as a `varstr`

In [ ]:
from exercises import serialize_version_payload

def test_serialize_version_payload_services_dict():
    now = int(time.time()) - 10
    version_payload = serialize_version_payload(
        services_dict={'NODE_NETWORK': True, 'NODE_BLOOM': True}, 
        nonce=4, timestamp=now, start_height=50, user_agent=b'/buidl-army/')
    version_payload = read_version_payload(BytesIO(version_payload))
    assert version_payload['version'] == 70015
    assert version_payload['services'] == 5
    assert version_payload['timestamp'] == now
    assert version_payload['nonce'] == 4
    assert version_payload['start_height'] == 50
    assert version_payload['user_agent'] == b'/buidl-army/'
    print('Test passed!')
    
test_serialize_version_payload_services_dict()

# "Network Address" Type

![image](../images/network-address.png)

Recall that network addresses are composed of 4 fields: `timestamp`, `services`, `ip` and a `port`. The latter three are always present, but the `timestamp` is present in all circumstances except version messages -- so our serialization code will need a flag for this just like our deserialization code had.

Here's how we deserialized network messages in the last lesson:

In [ ]:
def read_address(stream, timestamp):
    r = {}
    if timestamp:
        r["timestamp"] = little_endian_to_int(stream.read(4))
    r["services"] = little_endian_to_int(stream.read(8))
    r["ip"] = bytes_to_ip(stream.read(16))
    r["port"] = big_endian_to_int(stream.read(2))
    return r

This takes a byte stream and a `has_timestamp` flag and returns a dictionary containing all the informtaion about this address. Once again, we need to do the opposite.

Honestly, this lesson is running a little long and this code won't be terribly useful to us. I think you get the idea. I'm just going to give you this one:

In [ ]:
import socket

IPV4_PREFIX = b"\x00" * 10 + b"\x00" * 2

def ip_to_bytes(ip):
    if ":" in ip:
        return socket.inet_pton(socket.AF_INET6, ip)
    else:
        return IPV4_PREFIX + socket.inet_pton(socket.AF_INET, ip)

def serialize_address(address, has_timestamp):
    result = b""
    if has_timestamp:
        result += int_to_little_endian(address['timestamp'], 8)
    result += int_to_little_endian(address['services'], 8)
    result += ip_to_bytes(address['ip'])
    result += int_to_big_endian(address['port'], 2)
    return result

### Exercise: Update `serialize_version_payload` to serialize network addresses

Remember, in the context of the version message the `has_timestamp` flag should be set to false.

In [ ]:
from exercises import serialize_version_payload, dummy_address

def test_serialize_version_payload_network_addresses():
    now = int(time.time()) - 10
    version_payload = serialize_version_payload(
        services_dict={'NODE_NETWORK': True, 'NODE_BLOOM': True}, 
        nonce=4, timestamp=now, start_height=50, user_agent=b'/buidl-army/')
    version_payload = read_version_payload(BytesIO(version_payload))
    assert version_payload['version'] == 70015
    assert version_payload['services'] == 5
    assert version_payload['timestamp'] == now
    assert version_payload['receiver_address'] == dummy_address
    assert version_payload['sender_address'] == dummy_address
    assert version_payload['nonce'] == 4
    assert version_payload['start_height'] == 50
    assert version_payload['user_agent'] == b'/buidl-army/'

    print('Test passed!')
    
test_serialize_version_payload_network_addresses()

# Serializing Messages

We can completely serialize version message payloads. Yay!

Now we just need to serialize the outer message structure itself. This will just be reversing the `read_message` function we created in the first lesson:

In [ ]:
# To refresh your memory ...

def read_message(stream):
    msg = {}
    magic = stream.read(4)
    if magic != NETWORK_MAGIC:
        raise Exception(f'Magic is wrong: {magic}')
    msg['command'] = stream.read(12).strip(b'\x00')
    payload_length = int.from_bytes(stream.read(4), 'little')
    checksum = stream.read(4)
    msg['payload'] = stream.read(payload_length)
    calculated_checksum = double_sha256(msg['payload'])[:4]
    if calculated_checksum != checksum:
        raise Exception('Checksum does not match')
    return msg

Here's an outline of how an inverse `serialize_message` function might work:

In [ ]:
def serialize_message(command, payload):
    result = b'magic bytes'
    result += b'command bytes'
    result += b'payload length bytes'
    result += b'checksum bytes'
    result += b'payload bytes'
    return result

### Exercise: Serialize Network Magic

Implement `serialize_message` in [exercises.py](/edit/3.%20Composing%20Version%20Messages/exercises.py#)

Try not to look up the magic bytes in a previous lesson's code. Pretend that doesn't exist. Instead go to the wiki and figure out how to interpret what the wiki says into python.

In [ ]:
def test_serialize_message(fields):
    from exercises import serialize_message
    m = serialize_message(command=b"version", payload=b"foo")
    if 'magic' in fields:
        assert m[0:4] == bytes([249, 190, 180, 217])
    if 'command' in fields:
        assert m[4:16] == b'version' + (5 * b'\x00')
    if 'length' in fields:
        assert m[16:20] == b'\x03\x00\x00\x00'
    if 'checksum' in fields:
        assert m[20:24] == b'\xc7\xad\xe8\x8f'
    if 'payload' in fields:
        assert m[24:] == b'foo'
    print("Test passed!")

In [ ]:
test_serialize_message(['magic'])

### Exercise: Serialize Command

Next we serialize the `command` field. Remember that it needs to be a fixed length (look up the exact length in the wiki) and you should right-pad with zero bytes (`b\x00`) until that length is reached.

In [ ]:
test_serialize_message(['magic', 'command'])

### Exercise: Serialize Payload Length

Let's assume that the `payload` argument is the actual bytes of the payload. You need to get the length of these bytes and encodes that as the correct number of bytes, with the correct byte order.

In [ ]:
test_serialize_message(['magic', 'command', 'length'])

### Exercise: Serialize Checksum

I suggest you get the test to pass using the `compute_checksum` function I imported for you in [exercises.py](/edit/3.%20Composing%20Version%20Messages/exercises.py#).

Once you get that working, try to implement the `compute_checksum` yourself. It's good practice to remember how it works!

In [ ]:
test_serialize_message(['magic', 'command', 'length', 'checksum'])

### Exercise: Serialize Payload

Lastly, add the payload itself

In [ ]:
test_serialize_message(['magic', 'command', 'length', 'checksum', 'payload'])

# Verack

In order to complete the [version handshake](https://en.bitcoin.it/wiki/Version_Handshake), we also need to read and write verack messages.

This is going to be easy, though -- verack messages don't contain any payload.

To read a verack message it suffices to just call `read_message(stream)` which will return `{"command": b"verack": "payload": b""}`. There's nothing left to do at this point. 

To serialize a verack message, we'd simply need to call `serialize_message(command=b"verack")` -- this would produce raw bytes that could be send over a socket. We don't pass in anything for `payload_dict` because veracks don't have a payload and the `payload_dict` parameter in `serialize_message` defaults to the empty dictionary.

With this out of the way, it's time for the main event ...

### Exercise: Version Handshake

In the cell below implememt `handshake`, a function which opens connections with Bitcoin full nodes.

In [ ]:
from exercises import *
from lib import read_message
from pprint import pprint

def handshake(address):
    sock = socket.create_connection(address, timeout=1)
    stream = sock.makefile("rb")

    # Step 1: our version message
    sock.sendall("OUR VERSION MESSAGE")
    print("Sent version")

    # Step 2: their version message
    peer_version = "READ THEIR VERSION MESSAGE HERE"
    print("Version: ")
    pprint(peer_version)

    # Step 3: their version message
    peer_verack = "READ THEIR VERACK MESSAGE HERE"
    print("Verack: ", peer_verack)

    # Step 4: our verack
    sock.sendall("OUR VERACK HERE")
    print("Sent verack")

    return sock, stream

In [ ]:
ADDRESS = ("46.19.137.74", 8333)

sock = handshake(ADDRESS)

There's no unittest accompanied here. You'll know it's working if it prints something like: 

```
Sent version
Version: 
{'nonce': 4067395353630939545,
 'receiver_address': {'ip': '::ffff:70.113.80.77',
                      'port': 53996,
                      'services': 0},
 'relay': 1,
 'sender_address': {'ip': '0.0.0.0', 'port': 0, 'services': 1037},
 'services': 1037,
 'start_height': 565000,
 'timestamp': 1551338287,
 'user_agent': b'/Satoshi:0.17.1(LearnMeABitcoin)/',
 'version': 70015}
Verack:  {'command': b'verack', 'payload': b''}
Sent verack
```

Another, even more fun way to test it is to write a function that listens on the stream returned by `handshake` and prints out every message received:

In [ ]:
def listen(address):
    sock, stream = handshake(address)
    while True:
        message = read_message(stream)
        print(f'Received message "{message["command"]}"')

In [ ]:
listen(ADDRESS)

Hopefully this exercise gives you a bit of a sense of what a full node does.

It starts by establishing connections to a few peers. It sends messages over these sockets to request data it is interested in, and has listeners like the one above which read messages one-by-one and route them to handler functions that know how to deal with messages of each type.

In the next lesson we'll extend this "listener" idea into a crawler. Whenever we connect to a new node we will request they send a list of their peers. We will visit each of those peers in turn!

As homework, see if you can figure out how to read the payload of some of the messages observed by the `handler`. The `addr` messages should be easy -- they just a list of network addresses. `inv` messages are far juicier! We'll deal with these more a little later on ...

If you would like to leave some feedback please drop it in [this Google form](https://goo.gl/forms/Fj3aDeTx4SXibFFC2)!